In [41]:
import blurbs

import numpy as np
import matplotlib.pyplot as plt
import csv
from functools import reduce
import re 
from collections import defaultdict

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

import pandas as pd

import math



# imports from my modules
import imp
import feature_generation
imp.reload(feature_generation)
import bullets
imp.reload(bullets)

# yay neural nets
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Normalization
from tensorflow.keras.activations import linear, relu, sigmoid


In [28]:
# We need to add 'zip' column to addr_df by merging with apt_df.
DIR = '/home/bryce/Projects/Data_Science/Apt_Prices/csvs/march8/'

address_file =  DIR + 'pd_address_info.csv' 
apt_file = DIR + 'pd_apt_info.csv'

addr_df = pd.read_csv(address_file, sep=';')
apt_df = pd.read_csv(apt_file, sep=';')


addrs_and_zips = apt_df[['address', 'zip']].drop_duplicates()
addr_df = addr_df.merge(addrs_and_zips, on='address')
apt_df.drop(columns=['zip'], inplace=True)
# The number of addresses might go down because we've thrown out apartments with missing data (say, no price)


In [29]:
# DIR = '/home/bryce/Projects/Data_Science/Apt_Prices/csvs/march8/'

# address_file =  DIR + 'pd_address_info.csv' 
# apt_file = DIR + 'pd_apt_info.csv'

# addr_df = pd.read_csv(address_file, sep=';')
# apt_df = pd.read_csv(apt_file, sep=';')

((apt_train, addr_train), (apt_test, addr_test)) = feature_generation.split_data(apt_df=apt_df, addr_df=addr_df)

print("addr_train shape: ", addr_train.shape)
print("addr_test shape: ", addr_test.shape)




apt_df shape before:  (3166, 7)
addr_df shape before:  (400, 8)
apt_df shape after:  (3151, 7)
addr_df shape after:  (292, 8)
addr_train shape:  (204, 8)
addr_test shape:  (88, 8)


/home/bryce/Projects/Data_Science/Apt_Prices/feature_generation.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  addr_df.drop_duplicates(subset=['address'], inplace=True)


In [37]:
feat_gen = feature_generation.FeatureGenerator(num_svd_components=10)
(X_train, y_train) = feat_gen.get_training_features(apt_df_train=apt_train, addr_df_train=addr_train)
(X_test, y_test) = feat_gen.get_testing_features(apt_df_test=apt_test, addr_df_test=addr_test)

dim training blurbs, before cleaning (204,)
dim cleaned_blurbs  (204,)
training svd matrix shape  (204, 30)
about to return the training_svd_df, with shape (204, 30)
We made it here!
But did we make it here?
columns in dataframe:  Index(['address', 'price', 'beds', 'baths', 'sq_ft', 'location',
       'units_in_building', 'url', 'title', 'bullets', 'blurb', 'latlng',
       'latlng_len', 'zip', 'combined_bullets', 'lat', 'lng', 'lat^2', 'lng^2',
       'lat * lng', 'svd_1', 'svd_2', 'svd_3', 'svd_4', 'svd_5', 'svd_6',
       'svd_7', 'svd_8', 'svd_9', 'svd_10', 'svd_11', 'svd_12', 'svd_13',
       'svd_14', 'svd_15', 'svd_16', 'svd_17', 'svd_18', 'svd_19', 'svd_20',
       'svd_21', 'svd_22', 'svd_23', 'svd_24', 'svd_25', 'svd_26', 'svd_27',
       'svd_28', 'svd_29', 'svd_30', 'Bullet_concept_0', 'Bullet_concept_1',
       'Bullet_concept_2', 'Bullet_concept_3', 'Bullet_concept_4',
       'Bullet_concept_5', 'Bullet_concept_6', 'Bullet_concept_7',
       'Bullet_concept_8', 'Bullet_co

Steps before we train the neural net

1) generate train/val/test data and inspect for sanity (DONE)
2) generate training and validation features, and inspect for sanity

In [38]:
all_col_names = list(X_train.columns)
print(all_col_names)

['address', 'beds', 'baths', 'sq_ft', 'location', 'units_in_building', 'url', 'title', 'bullets', 'blurb', 'latlng', 'latlng_len', 'zip', 'combined_bullets', 'lat', 'lng', 'lat^2', 'lng^2', 'lat * lng', 'svd_1', 'svd_2', 'svd_3', 'svd_4', 'svd_5', 'svd_6', 'svd_7', 'svd_8', 'svd_9', 'svd_10', 'svd_11', 'svd_12', 'svd_13', 'svd_14', 'svd_15', 'svd_16', 'svd_17', 'svd_18', 'svd_19', 'svd_20', 'svd_21', 'svd_22', 'svd_23', 'svd_24', 'svd_25', 'svd_26', 'svd_27', 'svd_28', 'svd_29', 'svd_30', 'Bullet_concept_0', 'Bullet_concept_1', 'Bullet_concept_2', 'Bullet_concept_3', 'Bullet_concept_4', 'Bullet_concept_5', 'Bullet_concept_6', 'Bullet_concept_7', 'Bullet_concept_8', 'Bullet_concept_9', 'furnished', 'zip_29401', 'zip_29403', 'zip_29405', 'zip_29406', 'zip_29407', 'zip_29412', 'zip_29414', 'zip_29464', 'zip_29466', 'zip_29485', 'zip_29492', 'property_type', 'property_type_house', 'property_type_townhouse', 'property_type_undefined', 'baths x baths', 'baths x zip_29401', 'baths x zip_29403

In [42]:
all_col_names = X_train.columns
cols = feature_generation.select_column_names(all_column_names=all_col_names,
                                              latlng = False,
                                              units_in_building = True,
                                              zip_feats = True,
                                              bullet_feats = True,
                                              property_type = True,
                                              furnished=True,
                                              blurb_feats = False,
                                              cross_term_res=[])
print(cols)
X_train_feats = X_train[cols]

normalizer = Normalization(axis=-1)
normalizer.adapt(X_train_feats)

['beds', 'baths', 'sq_ft', 'units_in_building', 'zip_29401', 'zip_29403', 'zip_29405', 'zip_29406', 'zip_29407', 'zip_29412', 'zip_29414', 'zip_29464', 'zip_29466', 'zip_29485', 'zip_29492', 'Bullet_concept_0', 'Bullet_concept_1', 'Bullet_concept_2', 'Bullet_concept_3', 'Bullet_concept_4', 'Bullet_concept_5', 'Bullet_concept_6', 'Bullet_concept_7', 'Bullet_concept_8', 'Bullet_concept_9', 'property_type_house', 'property_type_townhouse', 'property_type_undefined', 'furnished']


2023-03-18 09:43:11.748749: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-18 09:43:11.748768: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-18 09:43:11.748783: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (bryce-IdeaPad-5-14ITL05): /proc/driver/nvidia/version does not exist
2023-03-18 09:43:11.748967: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [55]:


model = tf.keras.models.Sequential([
    normalizer,
    Dense(10, activation='relu', name='L1', kernel_regularizer=tf.keras.regularizers.L2(0.15)),
    Dense(5, activation='relu', name='L2', kernel_regularizer=tf.keras.regularizers.L2(0.15)),
    Dense(1, activation='linear', name='L3')
], name = 'FirstModel')

model.compile(loss='mean_absolute_error',
              optimizer=tf.keras.optimizers.Adam(0.005))




In [ ]:
# Pick params systematically

layer_nodes = [(20, 10), (10, 5), (40), (40, 10), (20, 10, 5)]
lambdas = [.02, .05, .10, .15]
model_num = 0
for lambda_ in lambdas:
    for node_sequence in layer_nodes:
        model_num += 1
        layers = []
        layer_num = 0
        for node_count in node_sequence:
            layer_num += 1
            name = "model" + str(model_num) + "_layer" + str(layer_num) + "_lambda(" + str(lambda_) + ")"
            layer = Dense(node_count, activation='relu', name=name, kernel_regularizer=tf.keras.regularizers.L2(lambda_))
            layers.append(layer)
        layers.append(Dense(1, activation='linear'))
        model_name = "model_num(" + str(model_num) + ")_lambda(" + str(lambda_) + ")"
        model = model = tf.keras.models.Sequential([
            normalizer,
            




In [62]:
history = model.fit(X_train_feats, y_train, epochs=800, verbose=1)


Epoch 1/800
58/58 [==============================] - 0s 1ms/step - loss: 190.5940
Epoch 2/800
58/58 [==============================] - 0s 1ms/step - loss: 191.0109
Epoch 3/800
58/58 [==============================] - 0s 928us/step - loss: 191.3680
Epoch 4/800
58/58 [==============================] - 0s 897us/step - loss: 190.6969
Epoch 5/800
58/58 [==============================] - 0s 808us/step - loss: 192.0729
Epoch 6/800
58/58 [==============================] - 0s 842us/step - loss: 191.7302
Epoch 7/800
58/58 [==============================] - 0s 863us/step - loss: 190.8596
Epoch 8/800
58/58 [==============================] - 0s 872us/step - loss: 190.7120
Epoch 9/800
58/58 [==============================] - 0s 867us/step - loss: 190.2422
Epoch 10/800
58/58 [==============================] - 0s 847us/step - loss: 191.3082
Epoch 11/800
58/58 [==============================] - 0s 858us/step - loss: 191.5168
Epoch 12/800
58/58 [==============================] - 0s 860us/step - loss: 19

In [63]:
results_train = model.evaluate(X_train_feats, y_train)
results_test = model.evaluate(X_test[cols], y_test)
print("training results: ", results_train)
print("test results: ", results_test)



24/24 [==============================] - 0s 712us/step - loss: 310.2763
training results:  187.4052734375
test results:  310.27630615234375


In [55]:
yhat = model.predict(X_test[cols])
y_diff = y_test - yhat 

Y = pd.DataFrame(data=y_diff)

25/25 [==============================] - 0s 2ms/step


In [62]:
def get_diff_df():
    D = X_test
    D = D.merge(addr_test, on='address')
    D['y'] = y_test
    D['yhat'] = yhat 
    D['diff'] = D.y - D.yhat 
    D.sort_values(by='diff', inplace=True)
    D.to_csv('/tmp/D.csv', sep=';')
    return D

import time

def scroll_through_overestimates():
    D = get_diff_df()
    i = 0
    while True:
            row = D.iloc[i]
            print(row.address, row.url, "beds and baths: ", row.beds, row.baths, "price: ", row.y, "estimated: ", row.yhat)
            i += 1
            time.sleep(10)


def scroll_through_underestimates():
    D = get_diff_df()
    i = D.shape[0] - 1
    while True:
            row = D.iloc[i]
            print(row.address, row.url, "beds and baths: ", row.beds, row.baths, "price: ", row.y, "estimated: ", row.yhat)
            i -= 1
            time.sleep(10)


In [63]:
scroll_through_underestimates()

13 Logan St, Charleston, SC 29401  https://www.apartments.com/4-br-4-bath-house-13-logan-street-charleston-sc/rqbyr41/ beds and baths:  4 4.0 price:  20000 estimated:  6697.7485
15 West St, Charleston, SC 29401  https://www.apartments.com/15-west-st-charleston-sc/lcjrtpf/ beds and baths:  2 2.25 price:  6000 estimated:  3117.2612
15 West St, Charleston, SC 29401  https://www.apartments.com/15-west-st-charleston-sc/lcjrtpf/ beds and baths:  2 2.25 price:  6000 estimated:  3117.2612
1123 Blakeway St, Charleston, SC 29492  https://www.apartments.com/1123-blakeway-st-daniel-island-sc/wqdexbl/ beds and baths:  3 2.0 price:  5700 estimated:  3570.7751
21 George St, Charleston, SC 29401  https://www.apartments.com/21-george-st-charleston-sc-unit-410/lmxl6nd/ beds and baths:  2 2.0 price:  5250 estimated:  3135.413
21 George St, Charleston, SC 29401  https://www.apartments.com/21-george-st-charleston-sc-unit-410/lmxl6nd/ beds and baths:  2 2.0 price:  4500 estimated:  3037.1042
1 Percy St, Cha

KeyboardInterrupt: 